In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle 

#Import the Dataset
dataset = pd.read_csv('/content/gdrive/MyDrive/Machine Learning/EMG/ML_Pattern_Recognition_Data V2.csv')
X = dataset.iloc[:, 1:].values
y = dataset.iloc[:, 0].values

In [ ]:
print(X)

[[101  97  97 ... 104 104 104]
 [ 68  69  68 ... 120 120 120]
 [ 71  73  74 ... 122 121 121]
 ...
 [ 42  44  44 ...  41  41  40]
 [ 41  41  41 ...  40  40  40]
 [ 47  48  48 ...  44  45  44]]


In [ ]:
print(y)

['A' 'A' 'A' ... 'B' 'B' 'B']


In [ ]:
#Splitting the dataset into Training and Testing Set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
!pip install catboost
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

     |████████████████████████████████| 65.8MB 54kB/s 
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
# Training XG Boost on Training Set
from catboost import CatBoostClassifier
classifier = CatBoostClassifier()
classifier.fit(X_train, y_train)

Learning rate set to 0.07831
0:	learn: 1.6832037	total: 4.89s	remaining: 1h 21m 27s
1:	learn: 1.6120537	total: 8.29s	remaining: 1h 8m 57s
2:	learn: 1.5488450	total: 11.7s	remaining: 1h 4m 50s
3:	learn: 1.4909294	total: 15.1s	remaining: 1h 2m 32s
4:	learn: 1.4371667	total: 18.5s	remaining: 1h 1m 20s
5:	learn: 1.3899924	total: 21.9s	remaining: 1h 28s
6:	learn: 1.3507997	total: 25.3s	remaining: 59m 46s
7:	learn: 1.3163038	total: 28.7s	remaining: 59m 16s
8:	learn: 1.2782524	total: 32s	remaining: 58m 48s
9:	learn: 1.2486106	total: 35.4s	remaining: 58m 25s
10:	learn: 1.2192159	total: 38.8s	remaining: 58m 4s
11:	learn: 1.1937383	total: 42.1s	remaining: 57m 49s
12:	learn: 1.1704511	total: 45.6s	remaining: 57m 38s
13:	learn: 1.1455867	total: 49s	remaining: 57m 28s
14:	learn: 1.1241514	total: 52.4s	remaining: 57m 18s
15:	learn: 1.1016864	total: 55.7s	remaining: 57m 5s
16:	learn: 1.0813407	total: 59.1s	remaining: 56m 57s
17:	learn: 1.0622755	total: 1m 2s	remaining: 56m 47s
18:	learn: 1.0464484	to

In [ ]:
# Making a Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[15  0  3  2  3  2]
 [ 0 35  1  0  0  1]
 [ 8  0 21  1  2  1]
 [ 5  0  3 21 10  1]
 [ 4  0  2  5 18  3]
 [ 1  0  3  0  1 32]]


0.696078431372549

In [ ]:
#Applying K-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Learning rate set to 0.077889
0:	learn: 1.6796900	total: 11.2s	remaining: 3h 6m 52s
1:	learn: 1.6032491	total: 14.5s	remaining: 2h 43s
2:	learn: 1.5326190	total: 17.8s	remaining: 1h 38m 47s
3:	learn: 1.4714107	total: 21.1s	remaining: 1h 27m 41s
4:	learn: 1.4185508	total: 24.4s	remaining: 1h 21m 5s
5:	learn: 1.3729931	total: 27.8s	remaining: 1h 16m 37s
6:	learn: 1.3322521	total: 31.1s	remaining: 1h 13m 26s
7:	learn: 1.2904451	total: 34.4s	remaining: 1h 11m
8:	learn: 1.2528696	total: 37.6s	remaining: 1h 9m 2s
9:	learn: 1.2238251	total: 40.9s	remaining: 1h 7m 33s
10:	learn: 1.1947444	total: 44.2s	remaining: 1h 6m 17s
11:	learn: 1.1668698	total: 47.5s	remaining: 1h 5m 13s
12:	learn: 1.1465593	total: 50.9s	remaining: 1h 4m 21s
13:	learn: 1.1232824	total: 54.2s	remaining: 1h 3m 37s
14:	learn: 1.1027946	total: 57.5s	remaining: 1h 2m 58s
15:	learn: 1.0845218	total: 1m	remaining: 1h 2m 20s
16:	learn: 1.0675279	total: 1m 4s	remaining: 1h 1m 50s
17:	learn: 1.0502416	total: 1m 7s	remaining: 1h 1m 

KeyboardInterrupt: ignored

In [ ]:
#Saving Model Using Pickle
pickle.dump(classifier, open('/content/gdrive/MyDrive/Machine Learning/EMG/EMG_clf.pkl', 'wb'))

In [ ]:
%%writefile app.py
# Import the Libraries
# Work with Data - the main Python libraries
import streamlit as st
import numpy as np
import pandas as pd
from PIL import Image, ImageOps # Image processing
import pickle

# Visualization
import matplotlib.pyplot as plt

# Modeling and Prediction
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

#Create a title and a sub-title
st.write("""
# EMG Based Silent Speech Interface
Helps Predict Speech communicated by paralyzed individuals using  Machine Learning
""")

#Open and Display an Image
#image = Image.open('/content/gdrive/My Drive/Machine Learning Web Application/Diabetes Detection/Diabetes Detection.png')
#st.image(image, use_column_width=True) # caption = 'ML', 

# COMPUTATION
st.sidebar.header('User Input Features')

st.sidebar.markdown("""
[Example CSV input file](https://raw.githubusercontent.com/dataprofessor/data/master/penguins_example.csv)
""")

# Collects user input features into dataframe
uploaded_file = st.sidebar.file_uploader("Upload your input CSV file", type=["csv"])
if uploaded_file is not None:
  input_df = pd.read_csv(uploaded_file)
  
# Combines user input features with entire penguins dataset
# This will be useful for the encoding phase
  with st.spinner('Processing CSV.....'):
    letters_raw = pd.read_csv('/content/gdrive/MyDrive/Machine Learning/EMG/ML_Pattern_Recognition_Data V2.csv')
    
    st.subheader('Training Data')
    st.write(letters_raw)

    try:
      letters = letters_raw.drop(columns=['Letter'])
    except:
      letters = letters_raw

  df = pd.concat([input_df, letters],axis=0)

  df = df[:1] # Selects only the first row (the user input data)

  # Displays the user input features
  st.subheader('User Input features')

  st.write(df)

  with st.spinner('Loading Model.....'):
    # Reads in saved classification model
    load_clf = pickle.load(open('/content/gdrive/MyDrive/Machine Learning/EMG/EMG_clf.pkl', 'rb'))

  with st.spinner('Making Prediction.....'):
    # Apply model to make predictions
    prediction = load_clf.predict(df)
    prediction_proba = load_clf.predict_proba(df)


  st.subheader('Prediction')
  prediction = str(prediction)

  if prediction == '[[\'A\']]':
    n=0
    prediction = 'A'
  elif prediction == '[[\'B\']]':
    n=1
    prediction = 'Blank'
  elif prediction == '[[\'E\']]':
    n=2
    prediction = 'E'
  elif prediction == '[[\'I\']]':
    n=3
    prediction = 'I'
  elif prediction == '[[\'O\']]':
    n=4
    prediction = 'O'
  elif prediction == '[[\'U\']]':
    n=5
    prediction = 'U'

  prediction_proba = prediction_proba[0][n]
  prediction_proba = prediction_proba * 100
  prediction_proba = str(prediction_proba)+'%'

  st.success(prediction)

  st.subheader('Prediction Probability')
  st.success(prediction_proba)
else:
  st.success("Please Upload CSV file to Continue")
  while uploaded_file is not None:
    st.spinner('Waiting for CSV input....')

Overwriting app.py


In [ ]:
!pip install streamlit
!npm install localtunnel
!streamlit run app.py &>/dev/null&
!npx localtunnel --port 8501

^C
